## Regresión Lineal

Alternativas para programación de Regresion Lineal y obtencion de parametros.

In [114]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import seaborn as sns
import statsmodels.formula.api as smf
import faraway.datasets.galapagos
#
import warnings
warnings.filterwarnings('ignore')

**Info Dataset**

30 obs (islas) y 6 variables.

- Species: numero de especies encontrados en la isla (target)
- Area: superficie de la isla en km2
- Elevation: elevacion de la isla (metros)
- Nearest: distancia (km) desde Santa Cruz
- Adjacent: area adjacente a la isla

In [111]:
# Data
df = faraway.datasets.galapagos.load()
print(df.shape)
df.head()

(30, 6)


,Species,Area,Elevation,Nearest,Scruz,Adjacent
Baltra,58,25.09,346,0.6,0.6,1.84
Bartolome,31,1.24,109,0.6,26.3,572.33
Caldwell,3,0.21,114,2.8,58.7,0.78
Champion,25,0.10,46,1.9,47.4,0.18
Coamano,2,0.05,77,1.9,1.9,903.82


## **Modelo Lineal**

La simplicidad de programacion de un modelo lineal, utilizando la forma de R.

In [86]:
# Modelo lineal (estilo R)
modelo = smf.ols(formula = 'Species ~ Area + Elevation + Nearest + Scruz + Adjacent', data=df).fit()
modelo.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                Species   R-squared:                       0.766
Model:                            OLS   Adj. R-squared:                  0.717
Method:                 Least Squares   F-statistic:                     15.70
Date:                Tue, 19 Jul 2022   Prob (F-statistic):           6.84e-07
Time:                        14:28:45   Log-Likelihood:                -162.54
No. Observations:                  30   AIC:                             337.1
Df Residuals:                      24   BIC:                             345.5
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      7.0682     19.154      0.369      0.715     -32.464      46.601
Area          -0.0239      0.022     -1.068      0.296      -0.070       0.022
Elevation      0.3195      0.054      5.953      0.000       0.209       0.430
Nearest        0.0091      1.054      0.009      0.993      -2.166       2.185
Scruz         -0.2405      0.215     -1.117      0.275      -0.685       0.204
Adjacent      -0.0748      0.018     -4.226      0.000      -0.111      -0.038
==============================================================================
Omnibus:                       12.683   Durbin-Watson:                   2.476
Prob(Omnibus):                  0.002   Jarque-Bera (JB):               13.498
Skew:                           1.136   Prob(JB):                      0.00117
Kurtosis:                       5.374   Cond. No.                     1.90e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.9e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

**Preguntas**
- Todos los predictores son significativos para el modelo?
- Existe coliniedad?

Por el momento no son importantes estas preguntas.

## Modelo Lineal programado de Forma Matricial

In [93]:
# insertamos la columnas de 1
X = df.iloc[:,1:]
X.insert(0,'intercepto',1)

# constructor
XtXi = np.linalg.inv(X.T @ X)

# obtenemos los betas, aunque de una forma muy ineficiente (demasiado computo, debido a la operacion de trasponer matrix)
(XtXi @ X.T) @ df.Species

0    7.068221
1   -0.023938
2    0.319465
3    0.009144
4   -0.240524
5   -0.074805
dtype: float64

In [95]:
# una forma mas eficiente
np.linalg.solve(X.T @ X, X.T @ df.Species)

# donde np.linalg.solve(A,B)  === > Ax = b

array([ 7.06822071, -0.02393834,  0.31946476,  0.00914396, -0.24052423,
       -0.07480483])

In [96]:
# RMSE (error cuadratico medio)
np.sqrt(modelo.mse_resid)

60.97518837269359

## Formas de estimar los betas

**Estimacion de los betas por minimos cuadrados**


**Diferenciando respecto a $\beta$**
- $X^T X\beta = X^Ty$

**Ecuaciones normales**
- $\beta = (X^T X)^-1 X^Ty$
- $X\beta = X(X^TX)^-1 Xy$
- $y = Hy$

Donde $H = X(X^TX)^-1 X^T$

In [112]:
# dimension
Xmp = np.linalg.pinv(X)
Xmp.shape

(6, 30)

In [113]:
# obtencion de los betas
Xmp @ df.Species

array([ 7.06822071, -0.02393834,  0.31946476,  0.00914396, -0.24052423,
       -0.07480483])

**Estimacion de los betas por el metodo inverso de Moore-Penrose**

**QR** es una alternativa del Metodo Inverso de Moore-Penrose para la obtencion de los betas.

In [100]:
q,r = np.linalg.qr(X)

#calculamos
f = q.T @ df.Species
f

array([-466.84219318,  381.40557435,  256.25047255,    5.40764552,
       -119.49834019,  257.69436853])

In [102]:
# resolviendo f (metodo de sustition hacia atras), obtenemos los betas
sp.linalg.solve_triangular(r,f)

array([ 7.06822071, -0.02393834,  0.31946476,  0.00914396, -0.24052423,
       -0.07480483])

**Utilizando Statsmodels con QR**

In [104]:
# Utilizacion de esta forma en statsmodels

modelo_form = smf.ols(formula = 'Species ~ Area + Elevation + Nearest + Scruz + Adjacent', data=df)
modelo1 = modelo_form.fit(method="qr")
modelo1.params

Intercept    7.068221
Area        -0.023938
Elevation    0.319465
Nearest      0.009144
Scruz       -0.240524
Adjacent    -0.074805
dtype: float64

**Utilizando Scipy**

In [108]:
# metodo alternativo utilizando scipy

params,res,rnk,s= sp.linalg.lstsq(X,df['Species'])
params

array([ 7.06822071, -0.02393834,  0.31946476,  0.00914396, -0.24052423,
       -0.07480483])